Let's load it...

In [1]:
# These are the possible input files
!ls -lh ../data/RecSys

total 4274488
drwxr-xr-x@ 3 czarrar  staff    96B Sep  4 23:26 core-data-images
-rw-rw-r--@ 1 czarrar  staff    12M Oct  6  2019 core-data-test_rating.csv
-rw-rw-r--@ 1 czarrar  staff    28M Oct  6  2019 core-data-train_rating.csv
-rw-rw-r--@ 1 czarrar  staff   5.5M Oct  6  2019 core-data-valid_rating.csv
-rw-rw-r--@ 1 czarrar  staff   173M Oct  6  2019 core-data_recipe.csv
drwxr-xr-x@ 3 czarrar  staff    96B Sep  4 23:26 raw-data-images
-rw-rw-r--@ 1 czarrar  staff   158M Oct  6  2019 raw-data_interaction.csv
-rw-rw-r--@ 1 czarrar  staff   1.6G Oct  6  2019 raw-data_recipe.csv


In [2]:
# Load in packages to use throughout
import pandas as pd
import json

# Read In

This is a big file (1.6gig). The raw recipes file is nice since it gives the average ratings for each recipe. It is the ratings that we are interested in to figure out if the proximity model is working well.

In [3]:
recipes = pd.read_csv('../data/RecSys/raw-data_recipe.csv')
recipes.head()

,recipe_id,recipe_name,aver_rate,image_url,review_nums,ingredients,cooking_directions,nutritions,reviews
0,222388,Homemade Bacon,5.000000,https://images.media-allrecipes.com/userphotos...,3,pork belly^smoked paprika^kosher salt^ground b...,{'directions': u'Prep\n5 m\nCook\n2 h 45 m\nRe...,"{u'niacin': {u'hasCompleteData': False, u'name...","{8542392: {'rating': 5, 'followersCount': 11, ..."
1,240488,"Pork Loin, Apples, and Sauerkraut",4.764706,https://images.media-allrecipes.com/userphotos...,29,sauerkraut drained^Granny Smith apples sliced^...,{'directions': u'Prep\n15 m\nCook\n2 h 30 m\nR...,"{u'niacin': {u'hasCompleteData': False, u'name...","{3574785: {'rating': 5, 'followersCount': 0, '..."
2,218939,Foolproof Rosemary Chicken Wings,4.571429,https://images.media-allrecipes.com/userphotos...,12,chicken wings^sprigs rosemary^head garlic^oliv...,"{'directions': u""Prep\n20 m\nCook\n40 m\nReady...","{u'niacin': {u'hasCompleteData': True, u'name'...","{13774946: {'rating': 5, 'followersCount': 0, ..."
3,87211,Chicken Pesto Paninis,4.625000,https://images.media-allrecipes.com/userphotos...,163,focaccia bread quartered^prepared basil pesto^...,{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{1563136: {'rating': 5, 'followersCount': 0, '..."
4,245714,Potato Bacon Pizza,4.500000,https://images.media-allrecipes.com/userphotos...,2,red potatoes^strips bacon^Sauce:^heavy whippin...,{'directions': u'Prep\n20 m\nCook\n45 m\nReady...,"{u'niacin': {u'hasCompleteData': True, u'name'...","{2945555: {'rating': 5, 'followersCount': 6690..."


In [4]:
# This comes from 10_Recipe_Ingredients
recipes2 = pd.read_csv('../data/20_ingredients.csv').iloc[:,1:]
recipes2.head()

,recipe_id,recipe_name,ingredients
0,240488,"Pork Loin, Apples, and Sauerkraut",sauerkraut drained^Granny Smith apples sliced^...
1,218939,Foolproof Rosemary Chicken Wings,chicken wings^sprigs rosemary^head garlic^oliv...
2,87211,Chicken Pesto Paninis,focaccia bread quartered^prepared basil pesto^...
3,245714,Potato Bacon Pizza,red potatoes^strips bacon^Sauce:^heavy whippin...
4,218545,Latin-Inspired Spicy Cream Chicken Stew,skinless boneless chicken breast halves^diced ...


I have taken the ratings and the ingredients together.

In [5]:
recipes3 = pd.merge(recipes[['recipe_id', 'aver_rate', 'review_nums']], recipes2, on="recipe_id")

In [6]:
print(recipes.shape)
print(recipes2.shape)
print(recipes3.shape)

(49698, 9)
(45630, 3)
(45630, 5)


In [7]:
recipes3.head()

,recipe_id,aver_rate,review_nums,recipe_name,ingredients
0,240488,4.764706,29,"Pork Loin, Apples, and Sauerkraut",sauerkraut drained^Granny Smith apples sliced^...
1,218939,4.571429,12,Foolproof Rosemary Chicken Wings,chicken wings^sprigs rosemary^head garlic^oliv...
2,87211,4.625000,163,Chicken Pesto Paninis,focaccia bread quartered^prepared basil pesto^...
3,245714,4.500000,2,Potato Bacon Pizza,red potatoes^strips bacon^Sauce:^heavy whippin...
4,218545,4.605769,85,Latin-Inspired Spicy Cream Chicken Stew,skinless boneless chicken breast halves^diced ...


In [52]:
recipes3 = recipes3.rename(columns={'aver_rate': 'ave_rating', 'review_nums': 'num_reviews', 'recipe_name': 'name'})

In [53]:
recipes3.head()

,recipe_id,ave_rating,num_reviews,name,ingredients,img_path
0,240488,4.764706,29,"Pork Loin, Apples, and Sauerkraut","sauerkraut drained, Granny Smith apples sliced...",../data/RecSys/core_images/240488.jpg
1,218939,4.571429,12,Foolproof Rosemary Chicken Wings,"chicken wings, sprigs rosemary, head garlic, o...",../data/RecSys/core_images/218939.jpg
2,87211,4.625000,163,Chicken Pesto Paninis,"focaccia bread quartered, prepared basil pesto...",../data/RecSys/core_images/87211.jpg
3,245714,4.500000,2,Potato Bacon Pizza,"red potatoes, strips bacon, Sauce:, heavy whip...",../data/RecSys/core_images/245714.jpg
4,218545,4.605769,85,Latin-Inspired Spicy Cream Chicken Stew,"skinless boneless chicken breast halves, diced...",../data/RecSys/core_images/218545.jpg


## Check on ingredients

Right now the ingredients use ^ to separate the different ingredients. This doesn't look great later on when showing the data to people. I can simplify this by changing the ^ to a comma. An issue with this is that 2 of the recipes have ingredients with commas there anyway. For those ones, I can just replace the comma with a space.

In [28]:
# Find the commas
import numpy as np

any_commas = np.array([ x.find(',') for x in recipes3.ingredients ])
np.sum(any_commas > (-1))

2

In [27]:
for x in recipes3.ingredients[any_commas > (-1)]:
    print(x.split('^'))

['chicken', 'chicken broth', 'large carrots', 'celery', 'large turnip,cut into 1/4-inch dice', 'parsnips cut into 1/4-inch dice', 'leek chopped', 'onion', 'chopped fresh dill', 'matzo ball mix', 'eggs', 'vegetable oil', 'water', 'salt and ground black pepper to taste']
['Corn "Cream":', 'ears fresh white corn', 'chicken broth', 'Pasta:', 'olive oil', 'diced bacon', 'diced sweet red pepper', 'diced zucchini', 'salt and pepper to taste', 'cayenne pepper', 'cavatelli pasta', 'corn "cream,"', 'halved sweet cherry tomatoes', 'chopped Italian parsley', 'finely sliced fresh basil leaves', 'grated Parmigiano-Reggiano cheese']


In [34]:
for ind in np.nonzero(any_commas > (-1))[0]:
    recipes3.loc[ind,"ingredients"] = recipes3.loc[ind,"ingredients"].replace(",", " ")

In [35]:
any_commas = np.array([ x.find(',') for x in recipes3.ingredients ])
np.sum(any_commas > (-1))

0

Replace the ^ with ,

In [36]:
recipes3['ingredients'] = [ x.replace('^', ', ') for x in recipes3.ingredients ]

In [37]:
recipes3.head()

,recipe_id,ave_rating,num_reviews,recipe_name,ingredients
0,240488,4.764706,29,"Pork Loin, Apples, and Sauerkraut","sauerkraut drained, Granny Smith apples sliced..."
1,218939,4.571429,12,Foolproof Rosemary Chicken Wings,"chicken wings, sprigs rosemary, head garlic, o..."
2,87211,4.625000,163,Chicken Pesto Paninis,"focaccia bread quartered, prepared basil pesto..."
3,245714,4.500000,2,Potato Bacon Pizza,"red potatoes, strips bacon, Sauce:, heavy whip..."
4,218545,4.605769,85,Latin-Inspired Spicy Cream Chicken Stew,"skinless boneless chicken breast halves, diced..."


## Add on the image paths

In [50]:
img_paths = [ '../data/RecSys/core_images/{}.jpg'.format(x) for x in recipes3.recipe_id ] 
recipes3['img_path'] = img_paths
recipes3.head()

,recipe_id,ave_rating,num_reviews,recipe_name,ingredients,img_path
0,240488,4.764706,29,"Pork Loin, Apples, and Sauerkraut","sauerkraut drained, Granny Smith apples sliced...",../data/RecSys/core_images/240488.jpg
1,218939,4.571429,12,Foolproof Rosemary Chicken Wings,"chicken wings, sprigs rosemary, head garlic, o...",../data/RecSys/core_images/218939.jpg
2,87211,4.625000,163,Chicken Pesto Paninis,"focaccia bread quartered, prepared basil pesto...",../data/RecSys/core_images/87211.jpg
3,245714,4.500000,2,Potato Bacon Pizza,"red potatoes, strips bacon, Sauce:, heavy whip...",../data/RecSys/core_images/245714.jpg
4,218545,4.605769,85,Latin-Inspired Spicy Cream Chicken Stew,"skinless boneless chicken breast halves, diced...",../data/RecSys/core_images/218545.jpg


# Save

In [54]:
recipes3.to_csv('../data/30_ingredients+ave_ratings.csv')

array([-1, -1, -1, ..., -1, -1, -1])

In [17]:
s.find(',')

-1